<a href="https://colab.research.google.com/github/Tejnu/CSET-419---Generative-artificial-intelligence/blob/main/GENAI_4_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from collections import defaultdict

In [2]:
# -------------------------------
# Dataset
# -------------------------------
text = """
artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.

"""

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Dense, LayerNormalization,
    Dropout, MultiHeadAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# -------------------------------
# Preprocessing
# -------------------------------
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    tokens = text.split()
    return tokens

tokens = preprocess_text(text)

In [5]:
# -------------------------------
# Tokenization
# -------------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]

In [6]:
# -------------------------------
# Create Input Sequences
# -------------------------------
input_sequences = []

for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_len = max(len(seq) for seq in input_sequences)

input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [7]:
# -------------------------------
#  Positional Encoding
# -------------------------------
def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    return tf.cast(angle_rads, dtype=tf.float32)

In [8]:
# -------------------------------
# Transformer Encoder Block
# -------------------------------
def transformer_encoder(inputs, head_size, num_heads, ff_dim):
    attention = MultiHeadAttention(
        key_dim=head_size,
        num_heads=num_heads
    )(inputs, inputs)

    attention = Dropout(0.1)(attention)
    x = LayerNormalization(epsilon=1e-6)(inputs + attention)

    ffn = Dense(ff_dim, activation="relu")(x)
    ffn = Dense(inputs.shape[-1])(ffn)
    ffn = Dropout(0.1)(ffn)

    return LayerNormalization(epsilon=1e-6)(x + ffn)

In [9]:
# -------------------------------
# Build Transformer Model
# -------------------------------
embed_dim = 64
inputs = Input(shape=(max_len-1,))
embedding = Embedding(total_words, embed_dim)(inputs)

pos_encoding = positional_encoding(max_len-1, embed_dim)
x = embedding + pos_encoding

x = transformer_encoder(x, head_size=64, num_heads=2, ff_dim=128)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = Dense(total_words, activation="softmax")(x)

model = Model(inputs, outputs)

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [10]:
def top_k_sampling(preds, k=5):
    preds = np.squeeze(preds)
    indices = np.argsort(preds)[-k:]
    probs = preds[indices]
    probs = probs / np.sum(probs)
    return np.random.choice(indices, p=probs)

In [11]:
def generate_text(seed_textt, next_words):
    for _ in range(next_words):
        seq = tokenizer.texts_to_sequences([seed_textt])[0]
        seq = pad_sequences([seq], maxlen=max_len-1, padding='pre')

        preds = model.predict(seq, verbose=0)
        next_index = top_k_sampling(preds, k=5)

        for word, index in tokenizer.word_index.items():
            if index == next_index:
                seed_textt += " " + word
                break

    return seed_textt

In [15]:
# -------------------------------
# TRANSFORMER
# -------------------------------
seed_textt = "artificial intelligence"
print("Generated Text:")
print(generate_text(seed_textt, 15))

Generated Text:
artificial intelligence grading processing programming grading grading stories stories grading grading stories programming grading stories processing processing
